In [ ]:
pip install numpy keras SimpleITK scikit-learn deap h5py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 33.5 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import SimpleITK as sitk

# Function to apply intensity normalization to 3D volume
def intensity_normalize_3d(input_data):
    # Normalize the entire 3D volume
    min_val = np.min(input_data)
    max_val = np.max(input_data)
    normalized_data = (input_data - min_val) / (max_val - min_val)
    return normalized_data

def spatial_normalize_3d(input_data):
    try:
        fixed_image_path = "/content/mni_icbm152_t1_tal_nlin_asym_09b_hires.nii"
        fixed_image = sitk.ReadImage(fixed_image_path)

        # Ensure the fixed image has the same data type as input_data (64-bit float)
        fixed_image = sitk.Cast(fixed_image, sitk.sitkFloat64)

        input_image = sitk.GetImageFromArray(input_data)

        # Resample input_image to match the dimensions of fixed_image
        input_image = sitk.Resample(input_image, fixed_image.GetSize(), sitk.Transform(), sitk.sitkLinear)

        input_image.CopyInformation(fixed_image)  # Now copy information

        registration = sitk.ImageRegistrationMethod()
        registration.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
        registration.SetMetricSamplingStrategy(sitk.ImageRegistrationMethod.RANDOM)
        registration.SetMetricSamplingPercentage(0.01)
        registration.SetInterpolator(sitk.sitkLinear)
        registration.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
        registration.SetOptimizerScalesFromPhysicalShift()
        initial_transform = sitk.AffineTransform(fixed_image.GetDimension())
        registration.SetInitialTransform(initial_transform)
        registration.AddCommand(sitk.sitkIterationEvent, lambda: print(f"Iteration: {registration.GetOptimizerIteration()} - Metric Value: {registration.GetMetricValue()}"))
        final_transform = registration.Execute(fixed_image, input_image)

        resampler = sitk.ResampleImageFilter()
        resampler.SetReferenceImage(fixed_image)
        resampler.SetTransform(final_transform)
        warped_image = resampler.Execute(input_image)

        warped_data = sitk.GetArrayFromImage(warped_image)

        return warped_data
    except Exception as e:
        print(f"An error occurred in spatial_normalize_3d: {e}")
        return None

# Load your 3D dataset here, assuming it's stored in a folder with sub-folders (classes)
data_folder = "/content/drive/MyDrive/ADoutput_file_NIFTi"
output_folder = "/content/drive/MyDrive/Normalization_output_file2"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through your 3D dataset, preserving class structure, and apply normalization
for class_folder in os.listdir(data_folder):
    class_path = os.path.join(data_folder, class_folder)
    if os.path.isdir(class_path):
        output_class_folder = os.path.join(output_folder, class_folder)
        os.makedirs(output_class_folder, exist_ok=True)

        for file_name in os.listdir(class_path):
            if file_name.endswith(".npy"):
                file_path = os.path.join(class_path, file_name)
                output_path = os.path.join(output_class_folder, file_name)

                # Check if the output file already exists, if so, skip this image
                if os.path.exists(output_path):
                    print(f"Skipping {file_name} as it's already preprocessed.")
                    continue

                print(f"Loading data from {file_path}")
                image_data = np.load(file_path)

                # Apply intensity normalization
                print("Applying intensity normalization...")
                normalized_data = intensity_normalize_3d(image_data)

                if normalized_data is not None:
                    # Apply spatial normalization
                    print("Applying spatial normalization...")
                    spatial_normalized_data = spatial_normalize_3d(normalized_data)

                    if spatial_normalized_data is not None:
                        # Save the spatially normalized data in the class-specific output folder
                        np.save(output_path, spatial_normalized_data)
                        print(f"Saved normalized data to {output_path}")

# Now, your normalized 3D dataset is saved in the output folder with the same class structure.


In [ ]:
#import shutil

#directory_to_delete = "/content/Normalization_output_file/VeryMildDemented"  # Replace with the actual path

try:
    shutil.rmtree(directory_to_delete)
    print(f"Directory '{directory_to_delete}' and its contents have been deleted successfully.")
except Exception as e:
    print(f"An error occurred while deleting the directory: {e}")


Directory '/content/Normalization_output_file/VeryMildDemented' and its contents have been deleted successfully.


In [ ]:
pip install tensorflow

In [ ]:
pip install opencv-python

In [ ]:
from deap import base, creator, tools, algorithms

In [ ]:
!pip install tensorflow-gpu
